In [1]:
import requests
from bs4 import BeautifulSoup
resp = requests.get("https://www.nytimes.com/interactive/2019/08/19/us/politics/presidential-campaign-songs-playlists.html")
html_str = resp.text
html_str
document = BeautifulSoup(html_str, "html.parser")
title_tags = document.find_all("span", attrs={"class": "song-title"})

title_tags

# make a dictionary in the format {song title, artist}

song_dict = {} 

for title in title_tags:
    # get the string associated w/ the tag
    song_title = title.string
    # while we're looking at that title, look for the next sibling
    artist_name = title.find_next_sibling('span')
    # add it to the dict (and be sure to add only the string)
    song_dict[song_title] = artist_name.string

In [2]:
client_access_token = 'CPaur6mX_hj-zyIPUCGZf2KQdtB3JQ5iRpsh2cqMvqcS1Co_UDhuRTP16oUYUc4V' # define my client access token

url_dict = {}

for item in song_dict:
    song_title = item
    song_artist = song_dict[item]
        
    # Remember this is the URL for a search via the Genius API:
    genius_search_url = f'http://api.genius.com/search?q={song_title}&access_token={client_access_token}'

    # here's the API call
    resp = requests.get(genius_search_url)
    data = resp.json()
    
    # now search for match w/ artist 
    for song in data['response']['hits']:
        if song['result']['primary_artist']['name'] == song_artist:
            # if there's a match, get the url
            lyrics_url = song['result']['url']

            # add it to the url dict in format {song title, url}
            url_dict[song_title] = lyrics_url
 
            # break out of for loop since we've got a match 
            break

In [3]:
import re

for item in url_dict:
    song_title = item
    song_artist = song_dict[item]   # note that later in the semester, we'll learn
                                    # another method of storing multidimensional data
                                    # for now, though, we'll just keep these two dicts
    song_url = url_dict[item]  

    # get the contents of the lyrics page
    resp = requests.get(song_url) 
    html_str = resp.text

    # turn it into a BS object and get the lyrics div
    document = BeautifulSoup(html_str, "html.parser")
    lyrics = document.find('p').text
    
    # let's do some quick cleaning 
    cleaner_lyrics = re.sub("\[.*\]\n", "", lyrics) # remove square brackets and contents
    cleanest_lyrics = re.sub("^[\n]{3}","", cleaner_lyrics) # remove the three opening newlines

    # store it in a file

    # we'll use the final part of the URL as a filename 
    print(song_url)
    filename = song_url.replace("https://genius.com/","")
    print(filename)
    filename = filename.replace("-lyrics","") + ".txt"
    print(filename)
    
    # store the lyrics in a subdir called "lyrics"; 
    path = "lyrics/" + filename
    
    with open(path, "w") as file:
        file.writelines(cleanest_lyrics)

https://genius.com/Aretha-franklin-respect-lyrics
Aretha-franklin-respect-lyrics
Aretha-franklin-respect.txt
https://genius.com/Dolly-parton-9-to-5-lyrics
Dolly-parton-9-to-5-lyrics
Dolly-parton-9-to-5.txt
https://genius.com/Ben-e-king-stand-by-me-lyrics
Ben-e-king-stand-by-me-lyrics
Ben-e-king-stand-by-me.txt
https://genius.com/Mary-j-blige-work-that-lyrics
Mary-j-blige-work-that-lyrics
Mary-j-blige-work-that.txt
https://genius.com/Panic-at-the-disco-high-hopes-lyrics
Panic-at-the-disco-high-hopes-lyrics
Panic-at-the-disco-high-hopes.txt
https://genius.com/The-ojays-love-train-lyrics
The-ojays-love-train-lyrics
The-ojays-love-train.txt
https://genius.com/Diana-ross-im-coming-out-lyrics
Diana-ross-im-coming-out-lyrics
Diana-ross-im-coming-out.txt
https://genius.com/Funkadelic-one-nation-under-a-groove-lyrics
Funkadelic-one-nation-under-a-groove-lyrics
Funkadelic-one-nation-under-a-groove.txt
https://genius.com/Neil-young-rockin-in-the-free-world-electric-lyrics
Neil-young-rockin-in-the

In [4]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /data/users/zfang36/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
with open("./lyrics/Beyonce-love-on-top.txt", "r") as file:
    lyrics1 = file.read()

In [6]:
with open("./lyrics/Alicia-keys-girl-on-fire-inferno-version.txt", "r") as file:
    lyrics2 = file.read()

In [7]:
stanzas1 = lyrics1.split('\n\n')

In [8]:
for stanza1 in stanzas1:
    print(stanza1)
    scores1 = sid.polarity_scores(stanza1)
    for key in scores1:
        print('{0}: {1}, '.format(key, scores1[key]), end='')
    print()

Bring the beat in
neg: 0.0, neu: 1.0, pos: 0.0, compound: 0.0, 
Honey, honey
I can see the stars all the way from here
Can't you see the glow on the window pane?
I can feel the sun whenever you're near
Every time you touch me, I just melt away
neg: 0.0, neu: 1.0, pos: 0.0, compound: 0.0, 
Now everybody asks me why I'm smiling out from ear to ear
(They say love hurts) But I know
(It's gonna take a little work) Oh
Nothing's perfect, but it's worth it
After fighting through my tears
And finally you put me first
neg: 0.112, neu: 0.701, pos: 0.186, compound: 0.4019, 
Baby, it's you, you're the one I love
You're the one I need, you're the only one I see
Come on, baby, it's you
You're the one that gives your all
You're the one I can always call
When I need you, make everything stop
Finally, you put my love on top
neg: 0.04, neu: 0.776, pos: 0.184, compound: 0.8402, 
Ooh, come on, baby!
You put my love on top, top, top, top, top
You put my love on top
Ooh, come on, baby!
You put my love on top

In [9]:
stanzas2 = lyrics2.split('\n\n')
for stanza2 in stanzas2:
    print(stanza2)
    scores3 = sid.polarity_scores(stanza1)
    for key in scores3:
        print('{0}: {1}, '.format(key, scores3[key]), end='')
    print()

Spirit of Marilyn callin' me, audibly
Bawling, she said that she would never leave, continued to torture me
Telling me to come with her, underneath my comforter
And she brought a gun with her, pills and some rum with her
Took me on the balcony, telling me to jump with her
Yeah I'm in the ghost, but I ain't doing stunts with her
I ain't trying to be that, haters wanna see that
But I got 'em aggy, 'cause I win the gold like Gabby
neg: 0.032, neu: 0.783, pos: 0.186, compound: 0.9889, 
She's just a girl, and she's on fire
Hotter than a fantasy, lonely like a highway
She's living in a world, and it's on fire
Feeling the catastrophe, but she knows she can fly away
neg: 0.032, neu: 0.783, pos: 0.186, compound: 0.9889, 
Oh, she got both feet on the ground
And she's burning it down
Oh, she got her head in the clouds
And she's not backing down
neg: 0.032, neu: 0.783, pos: 0.186, compound: 0.9889, 
This girl is on fire
This girl is on fire
She's walking on fire
This girl is on fire
neg: 0.032, ne

Part 2:

a. I chose these two songs since I've listened to them since 7th grade. Both of them have a strong and straightforward emotion. 'Love on Top' shows the woman's love for a man strongly. 'Girl on Fire' demonstrates the character of a brave girl. Therefore, they're easy for sentiment analysis to produce accurate outcomes.

b. They belong to 'hits'.

c. I analyze the songs according to stanza. I think a stanza is long enough to tell a story or express certain emotions. If we simply analyze by word or sentence, it's easy to miss the context and produce inaccurate analysis. 

d.

For 'Love on Top'

Ooh, come on, baby!
You put my love on top, top, top, top, top
You put my love on top
Ooh, come on, baby!
You put my love on top, top, top, top, top
You put my love on top
neg: 0.0, neu: 0.381, pos: 0.619, compound: 0.9861,

I think this stanza score accurately reflect the contents of that chunk. Although the stanza's lyrics is rather straightforward, it uses words like 'baby','top','love' which demonstrates that one person loves another strongly and deeply. However, it's also not that positive since the lyrics don't tell much about strong opinions. The result shows no negative score and majority of positive score. So I think the score reflects thee content well. 

For 'Girl on Fire'

She's just a girl, and she's on fire
Hotter than a fantasy, lonely like a highway
She's living in a world, and it's on fire
Feeling the catastrophe, but she knows she can fly away
neg: 0.032, neu: 0.783, pos: 0.186, compound: 0.9889,

I think this score also reflects the content well. This chunk is only describing the girl, about her characteristics, condition and ability. 'Hotter than fantasy' is positive while 'lonely like a highway' is rather negative. Besides, this chunk does not show strong opinions, so it's overall relatively neutral. The score reflects the content is mostly neutral, which fits the content and meaning of this chunk. 

e. 

For 'Love on Top'

Nothing's perfect, but it's worth it
After fighting through my tears
And finally you put me first
neg: 0.112, neu: 0.701, pos: 0.186, compound: 0.4019, 

I think this stanza's score doesn't reflect the content accurately. The lyrics of this stanza uses contrast to strengthen the uneasy happiness and positive feelings. However, the sentiment analysis scores it mostly as neutral. 

For 'Girl on Fire'

Looks like a girl, but she's a flame
So bright, she can burn your eyes
Better look the other way
You can try, but you'll never forget her name
She's on top of the world
Hottest of the hottest girls say
neg: 0.032, neu: 0.783, pos: 0.186, compound: 0.9889,

The stanza uses simile and intends to highlight that the girl is bright, unforgettable, even 'on top of the world', which are very positive characterizations. However, sentiment analysis just perceives it as neutral. 

f. I think the candidates who chose these songs had a tendency for songs with strong emotion and straightforward ideas, which make sentiment analysis easier. 

g. This exercise teaches me that sentiment analysis can process information really quickly. It could also capture straightforward opinions and ideas accurately. However, for complex sentences and rhetoric, it doesn't perform well each time.